**I.MÔ TẢ BÀI TOÁN :**

Bài toán đề xuất giá mặt hàng : Việc định giá sản phẩm hiện nay khó khăn trên quy mô lớn do các mặt hàng được định giá theo nhiều phương diện khác nhau ví dụ như quần áo định giá sản phẩm theo mùa ,còn giá của các sản phẩm điện tử thì định giá theo thông số kĩ thuật của nó . Qua đây , Mecari muốn chúng ta xây dựng thuật toán để tự động đề xuất giá sản phẩm phù hợp cho người bán dựa trên những thông tin mà họ cung cấp như tên danh mục sản phẩm, tên thương hiệu và tình trạng mặt hàng.

-Giải nén dữ liệu :

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
# !p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z
# !unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
#Import thư viện :
import numpy as np 
import pandas as pd 
import time
import lzma
import os
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import re

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

-Kiểm tra dữ liệu đầu vào : 

In [ ]:
#đọc dữ liệu đầu vào :
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test_stg2.tsv', sep='\t')
print("train data: ",train.shape)
print("test data: ",test.shape)

In [ ]:
#Thông tin tệp train 
train.info()

In [ ]:
# danh sách 10 mặt hàng đầu tiên : 
train.head(10)

In [ ]:
# kiểm tra thong tin ở tệp test 
test.info()

In [ ]:
test.head(10)

-Tệp train có 1482535 sản phẩm còn ở tệp test có 3460725 sản phẩm 

-Ta có thể thấy được ở tệp train có thêm cột price so với tệp test

-Trong danh mục mặt hàng có nhiều mặt hàng có dữ liệu null ở cột brand_name (được hiển thị là NaN)

**III.XỬ LÍ DỮ LIỆU ĐẦU VÀO**

1.Kiểm tra dữ liệu :

In [ ]:
# Thống kê mặt hàng theo nhãn hiệu:
unique_brands = train['brand_name'].value_counts()
print("Tổng số nhãn hiệu là {}".format(len(unique_brands)))
plt.figure(figsize=(20, 5))
sns.barplot(unique_brands.index[1:10], unique_brands[1:10])
plt.xlabel('Tên thương hiệu')
plt.ylabel('Số sản phẩm')
plt.show()

In [ ]:
#Đánh giá về giá bán :
fig, ax = plt.subplots(figsize=(10,6))
ax.hist(train.price,bins = 30, range = [min(train.price), max(train.price)],label="Price")
ax.set_xlabel('Price',fontsize=15)
plt.show()

-Ta có nhiều mặt hàng giá tập trung dưới 500$ ,phân phối về giá bị lệch phải 

-Sử dụng hàm đồng biến đưa về phân bố chuẩn :

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.hist(np.log1p(train['price']),bins = 30, range = [min(np.log1p(train['price'])), max(np.log1p(train['price']))],label="Price")
ax.set_xlabel('log(Price)',fontsize=15)
plt.show()

-Kiểm tra điều kiện của sản phẩm:

In [ ]:
count = train['item_condition_id'].value_counts()
plt.figure(figsize=(10, 5))
sns.barplot(count.index[:5], count[:5])
plt.ylabel('Số lượng sản phẩm')
plt.xlabel('Điều kiện sản phẩm')

-Kiểm tra phí ship :

In [ ]:
count = train['shipping'].value_counts()
plt.figure(figsize=(10, 5))
sns.barplot(count.index[:2], count[:2])
plt.ylabel('Số lượng')
plt.xlabel('Phí ship')

-Phần lớn các sp không có phí ship chiếm khoảng 55% còn lại 45 % không có ship

2.Xử lí dữ liệu :

-Ở cột category_name có các mục phân cách bởi dấu '/' nên chúng ta sẽ chia thành 3 cột để dễ xử lí

In [ ]:
def split_cat(text):
    try: return text.split("/")
    except: return ("None", "None", "None")
train['category_1'], train['category_2'], train['category_3'] = \
zip(*train['category_name'].apply(lambda x: split_cat(x)))
test['category_1'], test['category_2'], test['category_3'] = \
zip(*test['category_name'].apply(lambda x: split_cat(x)))

#coi lại dữ liệu sau khi tách
train.head(5)


-check sữ liệu trong tập train và test có null không 

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

-Ta thấy dữ liệu null ở tệp train và test tập trung ở cột category_name ,brand_name và item_description

-Xử lí dữ liệu null:

In [ ]:
def handle_missing(dataset):
    dataset.brand_name.fillna(value="None", inplace=True)
    dataset.item_description.fillna(value="None", inplace=True)
    dataset.category_name.fillna(value="None", inplace=True)
    return (dataset)

train = handle_missing(train)
test = handle_missing(test)

In [ ]:
#kiểm tra dữ liệu còn null không 
train.isnull().sum()

**IV.THỰC NGHIỆM** 

In [ ]:
#Nhóm tên sản phẩm theo giá của nó 
def create_price_d(grouped, operation_name, orig_colname):
    price_d = {}
    for index, row in grouped.iterrows():
        price_d[row[orig_colname]] = row['price']
    return {orig_colname + "_" + operation_name: price_d}


# Tạo ra dữ liệu để tra cứu giá theo tên sản phẩm
price_d = {}    
for col in ['brand_name','category_1','category_2','category_3' ,'category_name']:
    price_d.update(create_price_d(train.groupby(col, as_index=False).min(), 'min', col))
    price_d.update(create_price_d(train.groupby(col, as_index=False).median(), 'median', col))
    price_d.update(create_price_d(train.groupby(col, as_index=False).mean(), 'mean', col))
    price_d.update(create_price_d(train.groupby(col, as_index=False).max(), 'max', col))

    
#Đưa các cột chứa giá sản phẩm vào data
def price_dt(dt, pricedicts):
    for col in ['brand_name','category_1','category_2','category_3','category_name']:
        for oper in ['_min', '_median', '_mean', '_max']:
            dt[col + oper] = dt[col].map(price_d[col + oper])
            dt[col + oper] = dt[col + oper].fillna(price_d[col + oper].get('Unknown', 5))

price_dt(train, price_dt)
price_dt(test, price_dt)

In [ ]:
train.head(5)

In [ ]:
col = ['item_condition_id', 
       'shipping', 
      'brand_name_min', 
       'brand_name_max', 
       'brand_name_median', 
       'brand_name_mean', 
      'category_name_min', 
       'category_name_max', 
       'category_name_median', 
       'category_name_mean'
      ]

train_col = train[col]
train_pr = train[['price']]

test_col = test[col]

**LinearRegression**

Linear Regression được sử dụng để phân tích dự đoán. Hồi quy tuyến tính là một cách tiếp cận tuyến tính để mô hình hóa mối quan hệ giữa tiêu chí hoặc phản ứng vô hướng và nhiều yếu tố dự báo hoặc biến giải thích. Hồi quy tuyến tính tập trung vào phân phối xác suất có điều kiện của phản hồi cho các giá trị của các yếu tố dự đoán. Đối với hồi quy tuyến tính, có nguy cơ trang bị quá mức. Công thức của hồi quy tuyến tính là: Y = bX + A.


**Ridge Regression** 

Ridge Regression là một kỹ thuật để phân tích dữ liệu hồi quy nhiều lần. Khi xảy ra đa cộng tuyến, các ước lượng bình phương nhỏ nhất là không chệch. Một mức độ chệch được thêm vào các ước tính hồi quy và kết quả là hồi quy sườn núi làm giảm các sai số tiêu chuẩn.


**LightGBM**

LightGBM viết tắt của Light Gradient Boosting Machine, là mã nguồn mở xử lý thuật toán tăng cường độ dốc (Gradient Boosting) được phát triển bởi Microsoft. Gradient Boosting là một thuật toán xuất phát từ thuật toán Cây quyết định (Decision Tree), nó thực hiện việc xây dựng tuần tự nhiều Cây quyết định và tiến hành học tập.

In [ ]:
#Train mô hình sử dụng linear regression
regr = linear_model.LinearRegression()
regr.fit(train_col, train_pr)

# Tạo ra dự đoán giá trên tập test
test['price'] = regr.predict(test_col)

#Đưa những dự đoán giá nhỏ hơn 0 về 0
test.loc[test.price < 0, 'price'] = 0


In [ ]:
submission_col = test[['test_id', 'price']]
submission_col.to_csv('submission.csv', index=False)
submission_col